In [1]:
# import packages
from collections import Counter
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics
import math
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# import dataset
df = pd.read_csv('../data/UCI_Credit_Card.csv')
df.head()